<a href="https://colab.research.google.com/github/narel14/AAI-501-Grp-7/blob/main/aai_501_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project For Applied Artificial Intellence Class 501
Strategic Planning of Cycling Infrastructure Through
Built Environment Analysis and Network Optimization

In [2]:
! pip install osmnx
import os
import networkx as nx
import osmnx as ox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 1.8 MB/s eta 0:00:00
